# Overview
- nb013の改良
- lgbmのハイパラ調整

# Const

In [1]:
NB = '014'
# N_SPLITS = 5
# SHOW_LOG = True
# VERBOSE = 10
# # VERBOSE = None
# EARLY_STOPPING_ROUNDS = 100


PATH_TRAIN = './../data/official/train.csv'
PATH_TEST = './../data/official/test.csv'
PATH_SAMPLE_SUBMITTION = './../data/official/atmaCup8_sample-submission.csv'
SAVE_DIR = f'../data/output_nb/nb{NB}/'

feat_train_only = ['JP_Sales', 'Global_Sales', 'NA_Sales', 'Other_Sales', 'EU_Sales']
feat_common = ['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher',
           'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Developer',
           'Rating']
feat_string = ['Platform', 'Genre', 'Publisher', 'Developer', 'Rating']
feat_cat = ['Platform', 'Genre', 'Rating']
feat_num = ['Year_of_Release', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count']
use_col = [
    'Platform',
    'Year_of_Release',
    'Genre',
    'Critic_Score',
    'Critic_Count',
    'User_Score',
    'User_Count',
    'Rating'
    ]

In [2]:
config_str = """
globals:
  seed: 2020
  show_log: False

feature:
    use_col: [
        'Platform',
        'Year_of_Release',
        'Genre',
        'Critic_Score',
        'Critic_Count',
        'User_Score',
        'User_Count',
        'Rating'
        ]

split:
  name: KFold
  params:
    n_splits: 5
    random_state: 2020
    shuffle: True

model:
    model_params:
        objective: 'regression'
        metric: 'rmse'
        n_estimators: 800
        max_depth: 55
        subsample: 0.5
        colsample_bytree: 0.9
        learning_rate: 0.02
        reg_alpha: 5.0
        reg_lambda: 5.0
        min_child_samples: 57
    
    train_params:
        categorical_feature: ['Platform', 'Genre', 'Rating']
        verbose: None
        early_stopping_rounds: 100
"""



# Import everything I need :)

In [3]:
import os
import time
import yaml
import types
import builtins
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import optuna

from lightgbm import LGBMRegressor 

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split, KFold

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


# My function

In [4]:
# noglobal

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

In [5]:
@noglobal
def metric(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred) ** .5

@noglobal
def preprocess_User_Score(df):
    '''
    - tbdをnanにする
    - stringをfloatにする
    '''
    mask = df.User_Score.values == 'tbd'
    df.User_Score[mask] = np.nan
    df.User_Score = df.User_Score.values.astype(float)
    return df

@noglobal
def string_encode(df_trn, df_te, cols):
    '''
    - np.nanがあれば、'nan'に置き換える
    - label encodingする
    '''
    df = pd.concat([df_trn, df_te], axis=0).copy()
    df[cols] = df[cols].replace(np.nan, 'nan')
    df_trn[cols] = df_trn[cols].replace(np.nan, 'nan')
    df_te[cols] = df_te[cols].replace(np.nan, 'nan')
    for col in cols:
        le = LabelEncoder()
        le.fit(df[col])
        df_trn[col] = le.transform(df_trn[col])
        df_te[col] = le.transform(df_te[col])
    return df_trn, df_te

@noglobal
def df_preprocessing(df_trn, df_te, string_cols):
    df_trn = preprocess_User_Score(df_trn)
    df_te = preprocess_User_Score(df_te)
    df_trn, df_te = string_encode(df_trn, df_te, string_cols)
    return df_trn, df_te

In [6]:
@noglobal
def run_fold_lgbm(_X_trn, _y_trn, _X_val, _y_val, _X_te, model_config, show_log=True):
    model_params = model_config['model_params']
    train_params = model_config['train_params']
    
    # train
    model = LGBMRegressor(**model_params)
    model.fit(_X_trn, _y_trn.values[:, 0],
              eval_set=[(_X_trn, _y_trn), (_X_val, _y_val)],
              **train_params
             )
    
    # predict
    y_trn_pred = model.predict(_X_trn)
    y_val_pred = model.predict(_X_val)
    _y_test_pred = model.predict(_X_te)
    
    # postprocessiing
    y_trn_pred[y_trn_pred <= 1] = 1
    y_val_pred[y_val_pred <= 1] = 1
    y_trn_pred = np.expm1(y_trn_pred)  # exp を適用して 1 を引く
    y_val_pred = np.expm1(y_val_pred)  # exp を適用して 1 を引く
    _y_test_pred = np.expm1(_y_test_pred)  # exp を適用して 1 を引く
    
    if show_log:
        print(show_log)
        print(f'score train: {metric(np.expm1(_y_trn), y_trn_pred):.5f}')
        print(f'score valid: {metric(np.expm1(_y_val), y_val_pred):.5f}')
        print('')
    
    return y_trn_pred, y_val_pred, _y_test_pred

@noglobal
def run_lgbm(X, y, X_te, splitter, config):
    show_log = config['globals']['show_log']
    model_config = config['model']

    oof = np.zeros(len(X))
    y_test_pred = np.zeros(len(X_te))
    for fold_i, (idx_trn, idx_val) in enumerate(splitter.split(X)):
        if show_log:
            print(f'::Fold {fold_i+1}/{splitter.n_splits} start at {time.ctime()}::')
        X_trn, X_val = X.iloc[idx_trn, :], X.iloc[idx_val, :]
        y_trn, y_val = y.iloc[idx_trn], y.iloc[idx_val]
        X_trn = pd.DataFrame(X_trn, columns=X.columns)
        X_val = pd.DataFrame(X_val, columns=X.columns)

        # train
        y_trn_pred, y_val_pred, _y_test_pred = run_fold_lgbm(X_trn, y_trn, X_val, y_val, 
                                                             X_te, model_config, 
                                                             show_log=show_log)

        # result
        oof[idx_val] = y_val_pred
        y_test_pred += _y_test_pred / splitter.n_splits


    y_test_pred[y_test_pred <= 1] = 1
    return oof, y_test_pred

# Preparation

set

In [7]:
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

config = yaml.safe_load(config_str)

<br>

load dataset

In [8]:
train = pd.read_csv(PATH_TRAIN)
test = pd.read_csv(PATH_TEST)
ss = pd.read_csv(PATH_SAMPLE_SUBMITTION)

<br>

preprocess

In [9]:
train, test = df_preprocessing(train, test, feat_string)

In [10]:
X = train[use_col].copy()
y = train[['Global_Sales']].copy()
X_te = test[use_col].copy()

In [11]:
X

,Platform,Year_of_Release,Genre,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,26,NaN,0,74.0,17.0,7.9,22.0,2
1,26,NaN,0,78.0,22.0,6.6,28.0,2
2,19,NaN,0,73.0,5.0,7.4,10.0,2
3,0,NaN,0,NaN,NaN,NaN,NaN,8
4,26,NaN,0,76.0,8.0,7.8,13.0,2
...,...,...,...,...,...,...,...,...
8354,13,2016.0,11,78.0,57.0,8.0,569.0,8
8355,13,2016.0,11,NaN,NaN,NaN,NaN,8
8356,20,2017.0,0,NaN,NaN,NaN,NaN,8
8357,18,2017.0,7,NaN,NaN,NaN,NaN,8


In [12]:
# RMSLE を RMSEとしいて解く
y = np.log1p(y)  # 1 を足してlog を適用

# Optuna

In [13]:
splitter = KFold(**config['split']['params'])

In [14]:
# 最小化したい関数
def objective(trial):
    print('')
    params = {
#         'objective': 'regression',
#         'metric': 'rms',
#         'n_estimators': 800,
#         'n_estimators': trial.suggest_int('n_estimators', 800, 800),
        'max_depth': trial.suggest_int('max_depth', 5, 300),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.1, 1.0, 0.1),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'reg_alpha': trial.suggest_discrete_uniform('reg_alpha', 1, 6, 1),
        'reg_lambda': trial.suggest_discrete_uniform('reg_lambda', 1, 6, 1),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
    }
    
    params['objective'] = 'regression'
    params['metric'] = 'rmse'
    params['n_estimators'] = 800
    
    
    config['model']['model_params'] = params
    
    oof, y_test_pred = run_lgbm(X, y, X_te, splitter, config)
    return metric(np.expm1(y), oof)

In [15]:
%%time
study = optuna.create_study()
study.optimize(objective, n_trials=3000)
print(study.best_trial)
print(f'\n best_params = {study.best_params}')


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[430]	valid_0's rmse: 1.01889	valid_1's rmse: 1.0556
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[595]	valid_0's rmse: 1.00186	valid_1's rmse: 1.09046
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[800]	valid_0's rmse: 1.00539	valid_1's rmse: 1.06417
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[375]	valid_0's rmse: 1.02445	valid_1's rmse: 1.04747
Training until validation scores don't improve for 100 rounds


[I 2020-12-07 11:19:32,633] Finished trial#0 with value: 1.062108883921379 with parameters: {'max_depth': 47, 'subsample': 0.2, 'colsample_bytree': 0.2, 'learning_rate': 0.033068146313901155, 'reg_alpha': 3.0, 'reg_lambda': 6.0, 'min_child_samples': 81}. Best is trial#0 with value: 1.062108883921379.


Did not meet early stopping. Best iteration is:
[800]	valid_0's rmse: 1.00668	valid_1's rmse: 1.05692

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[681]	valid_0's rmse: 0.869512	valid_1's rmse: 0.994576
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[699]	valid_0's rmse: 0.861479	valid_1's rmse: 1.02094
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[614]	valid_0's rmse: 0.876178	valid_1's rmse: 1.00781
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[618]	valid_0's rmse: 0.881683	valid_1's rmse: 0.972216
Training until validation scores don't improve for 100 rounds


[I 2020-12-07 11:19:36,461] Finished trial#1 with value: 1.0023730799484258 with parameters: {'max_depth': 125, 'subsample': 0.1, 'colsample_bytree': 0.8, 'learning_rate': 0.022483970433585007, 'reg_alpha': 3.0, 'reg_lambda': 5.0, 'min_child_samples': 13}. Best is trial#1 with value: 1.0023730799484258.


Early stopping, best iteration is:
[539]	valid_0's rmse: 0.877764	valid_1's rmse: 1.01565

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[124]	valid_0's rmse: 0.914863	valid_1's rmse: 0.996934
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[209]	valid_0's rmse: 0.870824	valid_1's rmse: 1.02844
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[225]	valid_0's rmse: 0.870514	valid_1's rmse: 1.01378
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[153]	valid_0's rmse: 0.9021	valid_1's rmse: 0.976834
Training until validation scores don't improve for 100 rounds


...

Early stopping, best iteration is:
[487]	valid_0's rmse: 0.877851	valid_1's rmse: 1.01565

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[499]	valid_0's rmse: 0.876524	valid_1's rmse: 0.997843
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[671]	valid_0's rmse: 0.847843	valid_1's rmse: 1.0224
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[657]	valid_0's rmse: 0.854979	valid_1's rmse: 1.00468
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[594]	valid_0's rmse: 0.867371	valid_1's rmse: 0.970355
Training until validation scores don't improve for 100 rounds


[I 2020-12-07 14:19:48,816] Finished trial#2998 with value: 1.0025316633135322 with parameters: {'max_depth': 126, 'subsample': 0.7000000000000001, 'colsample_bytree': 1.0, 'learning_rate': 0.025510040817031752, 'reg_alpha': 3.0, 'reg_lambda': 6.0, 'min_child_samples': 10}. Best is trial#1467 with value: 1.0012697397653987.


Early stopping, best iteration is:
[477]	valid_0's rmse: 0.870827	valid_1's rmse: 1.01661

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[382]	valid_0's rmse: 0.887524	valid_1's rmse: 0.994698
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[800]	valid_0's rmse: 0.82452	valid_1's rmse: 1.0241
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[379]	valid_0's rmse: 0.88605	valid_1's rmse: 1.00683
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[490]	valid_0's rmse: 0.871986	valid_1's rmse: 0.972862
Training until validation scores don't improve for 100 rounds


[I 2020-12-07 14:19:51,940] Finished trial#2999 with value: 1.0032996487568675 with parameters: {'max_depth': 136, 'subsample': 0.9, 'colsample_bytree': 0.8, 'learning_rate': 0.029673869928564245, 'reg_alpha': 1.0, 'reg_lambda': 6.0, 'min_child_samples': 12}. Best is trial#1467 with value: 1.0012697397653987.


Early stopping, best iteration is:
[493]	valid_0's rmse: 0.860458	valid_1's rmse: 1.01732
FrozenTrial(number=1467, value=1.0012697397653987, datetime_start=datetime.datetime(2020, 12, 7, 12, 47, 23, 707841), datetime_complete=datetime.datetime(2020, 12, 7, 12, 47, 27, 405614), params={'max_depth': 131, 'subsample': 0.6, 'colsample_bytree': 0.9, 'learning_rate': 0.022862256818781214, 'reg_alpha': 1.0, 'reg_lambda': 6.0, 'min_child_samples': 10}, distributions={'max_depth': IntUniformDistribution(high=300, low=5, step=1), 'subsample': DiscreteUniformDistribution(high=1.0, low=0.1, q=0.1), 'colsample_bytree': DiscreteUniformDistribution(high=1.0, low=0.1, q=0.1), 'learning_rate': LogUniformDistribution(high=0.1, low=0.001), 'reg_alpha': DiscreteUniformDistribution(high=6, low=1, q=1), 'reg_lambda': DiscreteUniformDistribution(high=6, low=1, q=1), 'min_child_samples': IntUniformDistribution(high=100, low=10, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1467, s